# <center>Extracting Spotify Ids</center>

In this notebook, Spotify URIs (Spotify ids) will be extracted using web developer tools from Spotify's Web API. With these Spotify URIs, we can retrieve song feature information such as the song's popularity, genre, loudness, etc.

More information here: https://developer.spotify.com/documentation/web-api/

## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

Start by reading the csv file we used for web scraping songs and artists.

In [2]:
music_data = pd.read_csv('wikipedia_scraper.csv', encoding='ISO-8859-1')

In [3]:
music_data.head()

,rank,title,artist,year
0,1,Theme from A Summer Place,Percy Faith,1960
1,2,He'll Have to Go,Jim Reeves,1960
2,3,Cathy's Clown,The Everly Brothers,1960
3,4,Running Bear,Johnny Preston,1960
4,5,Teen Angel,Mark Dinning,1960


## Spotify Web API

Before we can start using Spotify's Web API, we need to link Spotify to an application. I already created my own application on the website, and will save the application's client id and client secret to a file I created called *Spotify.txt*. Then I will save these information so that Spotify can authenticate access to their Web API.

In [5]:
file = 'Spotify.txt'

# store client id and client secret
with open(file,'r') as f: 
    f = f.read().splitlines()
    cid = f[0]
    secret = f[1]
    
# authenticate requests to the Spotify Web API
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

With authentication permitted, we will use artist name and song title from our dataframe to attempt and search for the corresponding Spotify URI. To get an understanding of how the web API works, I used the code below from https://developer.spotify.com/documentation/web-api/reference/search/search/. This code will create new columns in our existing dataframe for Spotify id and song popularity.

In [11]:
# Code used from: https://towardsdatascience.com/what-makes-a-song-great-part-2-e82a44be659c

def get_spotify_data(dataframe):
    """
    Takes a dataframe as input.
    Returns a list of track ids and a list of popularity scores from the Spotify API. 
    """
    from numpy import nan # we import np.nan to handle empty queries 
    track_ids = []
    popularities = []
    for [artist, song] in list(zip(dataframe['artist'], dataframe['title'])):
        try:
            song_data = sp.search(q='artist:' + artist + ' track:' + song, type='track')
            track_id = song_data['tracks']['items'][0]['id']
            popularity = song_data['tracks']['items'][0]['popularity']
            track_ids.append(track_id)
            popularities.append(popularity)
        except:
            track_ids.append(nan)
            popularities.append(nan)
    return track_ids, popularities
track_ids, popularities = get_spotify_data(music_data)
music_data['Spotify id'] = track_ids
music_data['Popularity'] = popularities
music_data

,rank,title,artist,year,Spotify id,Popularity
0,1,Theme from A Summer Place,Percy Faith,1960,6zwvB879PJSpTyFcg2wwnL,57.0
1,2,He'll Have to Go,Jim Reeves,1960,7dDE59NX0n466e705E8Itz,18.0
2,3,Cathy's Clown,The Everly Brothers,1960,1MA9StLzlFftLbuqOmoWij,52.0
3,4,Running Bear,Johnny Preston,1960,1RYznli2VNO7FCbW1Hq4KM,39.0
4,5,Teen Angel,Mark Dinning,1960,36NPEs4S7ik50NrlzaqoIJ,31.0
...,...,...,...,...,...,...
6096,96,More Than My Hometown,Morgan Wallen,2020,0eBXyY4SatzpE7opnzgXvz,83.0
6097,97,Lovin' on You,Luke Combs,2020,0nYvjcSlCgjcwogQAwIwNp,76.0
6098,98,Said Sum,Moneybagg Yo,2020,3sKz6Sd72K0ofPWcJPPk6H,75.0
6099,99,Slide,H.E.R. featuring YG,2020,NaN,NaN


In [13]:
music_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6101 entries, 0 to 6100
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   rank        6101 non-null   object 
 1   title       6101 non-null   object 
 2   artist      6101 non-null   object 
 3   year        6101 non-null   int64  
 4   Spotify id  4950 non-null   object 
 5   Popularity  4950 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 286.1+ KB


In [14]:
music_data['Spotify id'].isnull().sum()

1151

## Splitting the artist column

There are a lot of songs without a Spotify id (1151 total). One reason why the Spotify ids for these songs were not extracted is because the get_spotify_data function does not recognize artists in the artist column that have multiple artists. These data contain strings such as "featuring", "and", and "&", so we'll split the artists into multiple columns that contain the primary artist and featured artists.

In [15]:
# the last two songs in our dataframe are good examples where spotify ids weren't extracted
music_data.tail()

,rank,title,artist,year,Spotify id,Popularity
6096,96,More Than My Hometown,Morgan Wallen,2020,0eBXyY4SatzpE7opnzgXvz,83.0
6097,97,Lovin' on You,Luke Combs,2020,0nYvjcSlCgjcwogQAwIwNp,76.0
6098,98,Said Sum,Moneybagg Yo,2020,3sKz6Sd72K0ofPWcJPPk6H,75.0
6099,99,Slide,H.E.R. featuring YG,2020,NaN,NaN
6100,100,Walk Em Down,NLE Choppa featuring Roddy Ricch,2020,NaN,NaN


In [45]:
# initialize with empty artist lists
artist_list = []
feat_artist_list = []

def split_artists(dataframe):
    '''
    Check if the artist column has any other featured artists.
    Then split the artists into lists
    '''
    
    for index, row in dataframe.iterrows():
        # if Spotify id doesn't exist, split the artist information
        if(pd.isnull(row['Spotify id'])):
            # string may contain certain characters that indicate multiple artists, so split the data
            artist_list.append(row['artist'].replace(" & ", "_" ).replace(" featuring ","_").replace(" and ","_").replace(", ","_").replace(" with ","_").split("_")[0])
            
            # there may or not be a featured artist, so check if this is true
            try:
                feat_artist_list.append(row['artist'].replace(" & ", "_" ).replace(" featuring ","_").replace(" and ","_").replace(", ","_").replace(" with ","_").split("_")[1])
            except:
                # No other artists for the song
                feat_artist_list.append("None")
        else:
            # Spotify id already exists, no need to split artist information
            artist_list.append(row['artist'])
            feat_artist_list.append("None")                               

In [17]:
split_artists(music_data)

In [18]:
# Check main artists
artist_list[-5:]

['Morgan Wallen', 'Luke Combs', 'Moneybagg Yo', 'H.E.R.', 'NLE Choppa']

In [19]:
# Check featured/secondary artists
feat_artist_list[-5:]

['None', 'None', 'None', 'YG', 'Roddy Ricch']

In [20]:
# another look at our dataframe
music_data

,rank,title,artist,year,Spotify id,Popularity
0,1,Theme from A Summer Place,Percy Faith,1960,6zwvB879PJSpTyFcg2wwnL,57.0
1,2,He'll Have to Go,Jim Reeves,1960,7dDE59NX0n466e705E8Itz,18.0
2,3,Cathy's Clown,The Everly Brothers,1960,1MA9StLzlFftLbuqOmoWij,52.0
3,4,Running Bear,Johnny Preston,1960,1RYznli2VNO7FCbW1Hq4KM,39.0
4,5,Teen Angel,Mark Dinning,1960,36NPEs4S7ik50NrlzaqoIJ,31.0
...,...,...,...,...,...,...
6096,96,More Than My Hometown,Morgan Wallen,2020,0eBXyY4SatzpE7opnzgXvz,83.0
6097,97,Lovin' on You,Luke Combs,2020,0nYvjcSlCgjcwogQAwIwNp,76.0
6098,98,Said Sum,Moneybagg Yo,2020,3sKz6Sd72K0ofPWcJPPk6H,75.0
6099,99,Slide,H.E.R. featuring YG,2020,NaN,NaN


We will insert the list of artists we obtained from using our defined function to our dataframe, along with any possible featured artists into two columns respectively: artist, other artists. If there are no other artists, we will insert "None" to the "other artists" column.

In [21]:
music_data["artist"] = artist_list

In [22]:
music_data["other artists"] = feat_artist_list

In [23]:
# other artists column has been added
music_data

,rank,title,artist,year,Spotify id,Popularity,other artists
0,1,Theme from A Summer Place,Percy Faith,1960,6zwvB879PJSpTyFcg2wwnL,57.0,None
1,2,He'll Have to Go,Jim Reeves,1960,7dDE59NX0n466e705E8Itz,18.0,None
2,3,Cathy's Clown,The Everly Brothers,1960,1MA9StLzlFftLbuqOmoWij,52.0,None
3,4,Running Bear,Johnny Preston,1960,1RYznli2VNO7FCbW1Hq4KM,39.0,None
4,5,Teen Angel,Mark Dinning,1960,36NPEs4S7ik50NrlzaqoIJ,31.0,None
...,...,...,...,...,...,...,...
6096,96,More Than My Hometown,Morgan Wallen,2020,0eBXyY4SatzpE7opnzgXvz,83.0,None
6097,97,Lovin' on You,Luke Combs,2020,0nYvjcSlCgjcwogQAwIwNp,76.0,None
6098,98,Said Sum,Moneybagg Yo,2020,3sKz6Sd72K0ofPWcJPPk6H,75.0,None
6099,99,Slide,H.E.R.,2020,NaN,NaN,YG


In [24]:
# reorder the columns
music_data = music_data[['rank', 'title', 'artist', 'other artists', 'year', 'Spotify id', 'Popularity']]

In [25]:
music_data

,rank,title,artist,other artists,year,Spotify id,Popularity
0,1,Theme from A Summer Place,Percy Faith,None,1960,6zwvB879PJSpTyFcg2wwnL,57.0
1,2,He'll Have to Go,Jim Reeves,None,1960,7dDE59NX0n466e705E8Itz,18.0
2,3,Cathy's Clown,The Everly Brothers,None,1960,1MA9StLzlFftLbuqOmoWij,52.0
3,4,Running Bear,Johnny Preston,None,1960,1RYznli2VNO7FCbW1Hq4KM,39.0
4,5,Teen Angel,Mark Dinning,None,1960,36NPEs4S7ik50NrlzaqoIJ,31.0
...,...,...,...,...,...,...,...
6096,96,More Than My Hometown,Morgan Wallen,None,2020,0eBXyY4SatzpE7opnzgXvz,83.0
6097,97,Lovin' on You,Luke Combs,None,2020,0nYvjcSlCgjcwogQAwIwNp,76.0
6098,98,Said Sum,Moneybagg Yo,None,2020,3sKz6Sd72K0ofPWcJPPk6H,75.0
6099,99,Slide,H.E.R.,YG,2020,NaN,NaN


With the new artist column that we recreated, let's reattempt and get the Spotify ids along with the song popularity.

In [29]:
track_ids, popularities = get_spotify_data(music_data)
music_data['Spotify id'] = track_ids
music_data['Popularity'] = popularities
music_data

,rank,title,artist,other artists,year,Spotify id,Popularity
0,1,Theme from A Summer Place,Percy Faith,None,1960,6zwvB879PJSpTyFcg2wwnL,57.0
1,2,He'll Have to Go,Jim Reeves,None,1960,7dDE59NX0n466e705E8Itz,18.0
2,3,Cathy's Clown,The Everly Brothers,None,1960,1MA9StLzlFftLbuqOmoWij,52.0
3,4,Running Bear,Johnny Preston,None,1960,1RYznli2VNO7FCbW1Hq4KM,39.0
4,5,Teen Angel,Mark Dinning,None,1960,36NPEs4S7ik50NrlzaqoIJ,31.0
...,...,...,...,...,...,...,...
6096,96,More Than My Hometown,Morgan Wallen,None,2020,0eBXyY4SatzpE7opnzgXvz,83.0
6097,97,Lovin' on You,Luke Combs,None,2020,0nYvjcSlCgjcwogQAwIwNp,76.0
6098,98,Said Sum,Moneybagg Yo,None,2020,3sKz6Sd72K0ofPWcJPPk6H,75.0
6099,99,Slide,H.E.R.,YG,2020,2rTnVB1bvwxHtaIl4uVu7f,77.0


In [30]:
music_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6101 entries, 0 to 6100
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   rank           6101 non-null   object 
 1   title          6101 non-null   object 
 2   artist         6101 non-null   object 
 3   other artists  6101 non-null   object 
 4   year           6101 non-null   int64  
 5   Spotify id     5690 non-null   object 
 6   Popularity     5690 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 333.8+ KB


In [31]:
music_data['Spotify id'].isnull().sum()

411

## Dealing with the remaining, missing Spotify ids

We dropped the amount of null values from 1151 to 411. At this point, there's nothing else we can do with python to extract the remaining Spotify ids, so the rest must be done manually.

We will save the data frame as a csv file so we can add the Spotify ids manually in Excel.

In [32]:
music_data.to_csv('spotify_ids.csv')

After manually inserting Spotify ids and doing some cleaning in Excel, we import the csv file.

In [3]:
new_music_data = pd.read_csv('spotify_ids.csv', encoding='ISO-8859-1')

In [11]:
new_music_data

,Unnamed: 0,rank,title,artist,other artists,year,Spotify id,Popularity,Spotify id.1
0,0,1,Theme from A Summer Place,Percy Faith,None,1960,6zwvB879PJSpTyFcg2wwnL,57.0,6zwvB879PJSpTyFcg2wwnL
1,1,2,He'll Have to Go,Jim Reeves,None,1960,7dDE59NX0n466e705E8Itz,18.0,7dDE59NX0n466e705E8Itz
2,2,3,Cathy's Clown,The Everly Brothers,None,1960,1MA9StLzlFftLbuqOmoWij,52.0,1MA9StLzlFftLbuqOmoWij
3,3,4,Running Bear,Johnny Preston,None,1960,1RYznli2VNO7FCbW1Hq4KM,39.0,1RYznli2VNO7FCbW1Hq4KM
4,4,5,Teen Angel,Mark Dinning,None,1960,36NPEs4S7ik50NrlzaqoIJ,31.0,36NPEs4S7ik50NrlzaqoIJ
...,...,...,...,...,...,...,...,...,...
6096,6096,96,More Than My Hometown,Morgan Wallen,None,2020,0eBXyY4SatzpE7opnzgXvz,83.0,0eBXyY4SatzpE7opnzgXvz
6097,6097,97,Lovin' on You,Luke Combs,None,2020,0nYvjcSlCgjcwogQAwIwNp,76.0,0nYvjcSlCgjcwogQAwIwNp
6098,6098,98,Said Sum,Moneybagg Yo,None,2020,3sKz6Sd72K0ofPWcJPPk6H,75.0,3sKz6Sd72K0ofPWcJPPk6H
6099,6099,99,Slide,H.E.R.,YG,2020,2rTnVB1bvwxHtaIl4uVu7f,77.0,2rTnVB1bvwxHtaIl4uVu7f


In [6]:
new_music_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6101 entries, 0 to 6100
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     6101 non-null   int64  
 1   rank           6101 non-null   object 
 2   title          6101 non-null   object 
 3   artist         6101 non-null   object 
 4   other artists  6101 non-null   object 
 5   year           6101 non-null   int64  
 6   Spotify id     6098 non-null   object 
 7   Popularity     5690 non-null   float64
 8   Spotify id.1   6098 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 429.1+ KB


In [7]:
new_music_data.isnull().sum()

Unnamed: 0         0
rank               0
title              0
artist             0
other artists      0
year               0
Spotify id         3
Popularity       411
Spotify id.1       3
dtype: int64

In [8]:
new_music_data[new_music_data["Spotify id"].isnull()]

,Unnamed: 0,rank,title,artist,other artists,year,Spotify id,Popularity,Spotify id.1
2182,2182,82,It's Now or Never,John Schneider,None,1981,NaN,NaN,NaN
2297,2297,97,Goin' Down,Greg Guidry,None,1982,NaN,NaN,NaN
3772,3772,72,ESPN Presents The Jock Jam,Various Artists,None,1997,NaN,NaN,NaN


## Removing rows with missing information

There are still 3 Spotify ids that are missing. Since these are just a very small portion of our entire dataset, it should be okay to just drop these rows.

In [12]:
new_music_data = new_music_data.drop([new_music_data.index[2182],
                                      new_music_data.index[2297],
                                      new_music_data.index[3772]])

In [13]:
new_music_data

,Unnamed: 0,rank,title,artist,other artists,year,Spotify id,Popularity,Spotify id.1
0,0,1,Theme from A Summer Place,Percy Faith,None,1960,6zwvB879PJSpTyFcg2wwnL,57.0,6zwvB879PJSpTyFcg2wwnL
1,1,2,He'll Have to Go,Jim Reeves,None,1960,7dDE59NX0n466e705E8Itz,18.0,7dDE59NX0n466e705E8Itz
2,2,3,Cathy's Clown,The Everly Brothers,None,1960,1MA9StLzlFftLbuqOmoWij,52.0,1MA9StLzlFftLbuqOmoWij
3,3,4,Running Bear,Johnny Preston,None,1960,1RYznli2VNO7FCbW1Hq4KM,39.0,1RYznli2VNO7FCbW1Hq4KM
4,4,5,Teen Angel,Mark Dinning,None,1960,36NPEs4S7ik50NrlzaqoIJ,31.0,36NPEs4S7ik50NrlzaqoIJ
...,...,...,...,...,...,...,...,...,...
6096,6096,96,More Than My Hometown,Morgan Wallen,None,2020,0eBXyY4SatzpE7opnzgXvz,83.0,0eBXyY4SatzpE7opnzgXvz
6097,6097,97,Lovin' on You,Luke Combs,None,2020,0nYvjcSlCgjcwogQAwIwNp,76.0,0nYvjcSlCgjcwogQAwIwNp
6098,6098,98,Said Sum,Moneybagg Yo,None,2020,3sKz6Sd72K0ofPWcJPPk6H,75.0,3sKz6Sd72K0ofPWcJPPk6H
6099,6099,99,Slide,H.E.R.,YG,2020,2rTnVB1bvwxHtaIl4uVu7f,77.0,2rTnVB1bvwxHtaIl4uVu7f


After dropping the 3 rows, we now have no more rows with missing Spotify ids. We just need to deal with missing Popularity information.

In [14]:
new_music_data.isnull().sum()

Unnamed: 0         0
rank               0
title              0
artist             0
other artists      0
year               0
Spotify id         0
Popularity       408
Spotify id.1       0
dtype: int64

## Cleaning dataframe

But first, let's clean the columns with our new dataframe.

In [15]:
# Drop index column, as well as the old spotify id column
new_music_data = new_music_data.drop(['Unnamed: 0', 'Spotify id'], axis=1)

# Rename column
new_music_data = new_music_data.rename(columns={'Spotify id.1': 'Spotify id'})

# Reorder columns
new_music_data = new_music_data[['rank', 'title', 'artist', 'other artists', 'year', 'Spotify id', 'Popularity']]

In [17]:
new_music_data

,rank,title,artist,other artists,year,Spotify id,Popularity
0,1,Theme from A Summer Place,Percy Faith,None,1960,6zwvB879PJSpTyFcg2wwnL,57.0
1,2,He'll Have to Go,Jim Reeves,None,1960,7dDE59NX0n466e705E8Itz,18.0
2,3,Cathy's Clown,The Everly Brothers,None,1960,1MA9StLzlFftLbuqOmoWij,52.0
3,4,Running Bear,Johnny Preston,None,1960,1RYznli2VNO7FCbW1Hq4KM,39.0
4,5,Teen Angel,Mark Dinning,None,1960,36NPEs4S7ik50NrlzaqoIJ,31.0
...,...,...,...,...,...,...,...
6096,96,More Than My Hometown,Morgan Wallen,None,2020,0eBXyY4SatzpE7opnzgXvz,83.0
6097,97,Lovin' on You,Luke Combs,None,2020,0nYvjcSlCgjcwogQAwIwNp,76.0
6098,98,Said Sum,Moneybagg Yo,None,2020,3sKz6Sd72K0ofPWcJPPk6H,75.0
6099,99,Slide,H.E.R.,YG,2020,2rTnVB1bvwxHtaIl4uVu7f,77.0


## Splitting the artist column (again)

Again, let's split some of the data from the artist column as some may have been missed (particularly those that did have a Spotify id from early on).

In [18]:
artist_list = []
feat_artist_list = []

def split_artists(dataframe):
    '''
    Check if the artist column has featured artists, then split artists into lists
    '''
    for index, row in dataframe.iterrows():
        if(row['other artists'] == "None"):
            artist_list.append(row['artist'].replace(" & ", "_").replace(" featuring ","_").replace(" and ","_").replace(", ","_").replace(" with ","_").split("_")[0])
            
            #there may or not be a featured artist, so check if this is true
            try:
                feat_artist_list.append(row['artist'].replace(" & ", "_").replace(" featuring ","_").replace(" and ","_").replace(", ","_").replace(" with ","_").split("_")[1])
            except:
                feat_artist_list.append("None")
        else:
            artist_list.append(row['artist'])
            feat_artist_list.append(row['other artists'])                               

In [19]:
split_artists(new_music_data)

In [20]:
new_music_data["artist"] = artist_list
new_music_data["other artists"] = feat_artist_list

In [21]:
new_music_data

,rank,title,artist,other artists,year,Spotify id,Popularity
0,1,Theme from A Summer Place,Percy Faith,None,1960,6zwvB879PJSpTyFcg2wwnL,57.0
1,2,He'll Have to Go,Jim Reeves,None,1960,7dDE59NX0n466e705E8Itz,18.0
2,3,Cathy's Clown,The Everly Brothers,None,1960,1MA9StLzlFftLbuqOmoWij,52.0
3,4,Running Bear,Johnny Preston,None,1960,1RYznli2VNO7FCbW1Hq4KM,39.0
4,5,Teen Angel,Mark Dinning,None,1960,36NPEs4S7ik50NrlzaqoIJ,31.0
...,...,...,...,...,...,...,...
6096,96,More Than My Hometown,Morgan Wallen,None,2020,0eBXyY4SatzpE7opnzgXvz,83.0
6097,97,Lovin' on You,Luke Combs,None,2020,0nYvjcSlCgjcwogQAwIwNp,76.0
6098,98,Said Sum,Moneybagg Yo,None,2020,3sKz6Sd72K0ofPWcJPPk6H,75.0
6099,99,Slide,H.E.R.,YG,2020,2rTnVB1bvwxHtaIl4uVu7f,77.0


In [26]:
new_music_data.isnull().sum()

rank               0
title              0
artist             0
other artists      0
year               0
Spotify id         0
Popularity       408
dtype: int64

## Filling in the missing popularity data

Now let's fill in the missing data with popularity information using Spotify's Web API.

In [65]:
popularities = []

for id_ in new_music_data['Spotify id']:
    feature = sp.track(id_)['popularity']
    popularities.append(feature)

In [74]:
new_music_data['Popularity'] = popularities

In [77]:
new_music_data.isnull().sum()

rank             0
title            0
artist           0
other artists    0
year             0
Spotify id       0
Popularity       0
dtype: int64

There are no more missing values in our dataframe! Let's go ahead and save this into our csv file.

In [78]:
new_music_data.to_csv('spotify_ids.csv')